# Gold Price Prediction - Model Experiments

This notebook contains experiments with different LSTM architectures and hyperparameters for gold price prediction.

## Objectives
1. Compare different LSTM architectures
2. Optimize hyperparameters
3. Analyze model performance
4. Select best model for production

In [ ]:
# Import libraries
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Add project root to path
sys.path.append('..')

from src.utils import setup_logging, load_config
from src.data_preprocessing import GoldDataPreprocessor
from src.lstm_model import LSTMGoldPredictor
from src.model_trainer import ModelTrainer, HyperparameterTuner
from src.evaluation import ModelEvaluator
from src.visualization import Visualizer
from src.mlflow_integration import MLflowManager
from config.config import Config

# Setup
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')
%matplotlib inline
%load_ext autoreload
%autoreload 2

print("Libraries imported successfully!")

## 1. Configuration and Data Loading

In [ ]:
# Load configuration
config = load_config()
setup_logging()

# Initialize MLflow for experiment tracking
mlflow_manager = MLflowManager(config, "model_experiments")

# Load and preprocess data
preprocessor = GoldDataPreprocessor(config.data)
data_path = '../data/sample_data.json'

# Load data
df = preprocessor.load_data(data_path)
print(f"Loaded {len(df)} data points")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")

# Preprocess data
X_train, X_val, X_test, y_train, y_val, y_test, scalers = preprocessor.preprocess_data(df)

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")

## 2. Baseline Model Experiments

In [ ]:
# Define baseline architectures to test
architectures = [
    'simple_lstm',
    'stacked_lstm', 
    'bidirectional_lstm',
    'lstm_with_attention',
    'cnn_lstm',
    'encoder_decoder'
]

baseline_results = {}
evaluator = ModelEvaluator(config.evaluation)

print("Testing baseline architectures...")

for arch in architectures:
    print(f"\n=== Testing {arch} ===")
    
    with mlflow_manager.start_run(f"baseline_{arch}"):
        try:
            # Create model
            model_config = config.model
            model_config.architecture = arch
            
            model = LSTMGoldPredictor(model_config)
            
            # Train model
            trainer = ModelTrainer(config.training)
            
            history = trainer.train_model(
                model, 
                X_train, y_train,
                X_val, y_val,
                epochs=20  # Quick training for comparison
            )
            
            # Evaluate model
            y_pred = model.predict(X_test)
            metrics = evaluator.evaluate_predictions(y_test, y_pred)
            
            # Store results
            baseline_results[arch] = {
                'metrics': metrics,
                'model': model,
                'history': history
            }
            
            # Log to MLflow
            mlflow_manager.log_hyperparameters({
                'architecture': arch,
                'sequence_length': model_config.sequence_length,
                'lstm_units': model_config.lstm_units,
                'dropout': model_config.dropout
            })
            
            mlflow_manager.log_metrics(metrics)
            
            print(f"MAPE: {metrics['mape']:.2f}%")
            print(f"RMSE: {metrics['rmse']:.2f}")
            print(f"R²: {metrics['r2']:.3f}")
            
        except Exception as e:
            print(f"Error training {arch}: {e}")
            baseline_results[arch] = {'error': str(e)}

print("\nBaseline experiments completed!")

## 3. Compare Baseline Results

In [ ]:
# Create comparison DataFrame
comparison_data = []

for arch, results in baseline_results.items():
    if 'metrics' in results:
        metrics = results['metrics']
        comparison_data.append({
            'Architecture': arch,
            'MAPE (%)': metrics['mape'],
            'RMSE': metrics['rmse'],
            'MAE': metrics['mae'],
            'R²': metrics['r2'],
            'Direction Accuracy (%)': metrics.get('direction_accuracy', 0) * 100
        })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('MAPE (%)')

print("Architecture Comparison:")
print(comparison_df.round(3))

# Visualize comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# MAPE comparison
axes[0, 0].bar(comparison_df['Architecture'], comparison_df['MAPE (%)'])
axes[0, 0].set_title('MAPE Comparison')
axes[0, 0].set_ylabel('MAPE (%)')
axes[0, 0].tick_params(axis='x', rotation=45)

# RMSE comparison
axes[0, 1].bar(comparison_df['Architecture'], comparison_df['RMSE'])
axes[0, 1].set_title('RMSE Comparison')
axes[0, 1].set_ylabel('RMSE')
axes[0, 1].tick_params(axis='x', rotation=45)

# R² comparison
axes[1, 0].bar(comparison_df['Architecture'], comparison_df['R²'])
axes[1, 0].set_title('R² Comparison')
axes[1, 0].set_ylabel('R²')
axes[1, 0].tick_params(axis='x', rotation=45)

# Direction Accuracy comparison
axes[1, 1].bar(comparison_df['Architecture'], comparison_df['Direction Accuracy (%)'])
axes[1, 1].set_title('Direction Accuracy Comparison')
axes[1, 1].set_ylabel('Direction Accuracy (%)')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Find best architecture
best_arch = comparison_df.iloc[0]['Architecture']
print(f"\nBest performing architecture: {best_arch}")

## 4. Hyperparameter Optimization

In [ ]:
# Hyperparameter optimization for best architecture
print(f"Optimizing hyperparameters for {best_arch}...")

# Define hyperparameter search space
hyperparameter_space = {
    'lstm_units': [32, 64, 128],
    'dropout': [0.1, 0.2, 0.3],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [16, 32, 64],
    'sequence_length': [30, 60, 90]
}

# Initialize hyperparameter tuner
tuner = HyperparameterTuner(
    config.training,
    search_method='random',  # Use random search for speed
    n_trials=10  # Limited trials for notebook
)

with mlflow_manager.start_run(f"hyperparam_opt_{best_arch}"):
    # Run optimization
    best_params, best_score, optimization_history = tuner.optimize(
        X_train, y_train,
        X_val, y_val,
        hyperparameter_space,
        architecture=best_arch
    )
    
    print(f"\nBest hyperparameters: {best_params}")
    print(f"Best score: {best_score:.4f}")
    
    # Log best parameters
    mlflow_manager.log_hyperparameters(best_params)
    mlflow_manager.log_metrics({'best_validation_score': best_score})

# Visualize optimization history
if optimization_history:
    plt.figure(figsize=(10, 6))
    plt.plot(optimization_history)
    plt.title('Hyperparameter Optimization Progress')
    plt.xlabel('Trial')
    plt.ylabel('Validation Score')
    plt.grid(True)
    plt.show()

## 5. Final Model Training and Analysis

In [ ]:
# Complete experiment with final model training and comprehensive analysis
print("Training final optimized model and generating comprehensive analysis...")

# This cell contains the complete experimental workflow
# Results will be saved and visualized for production deployment decision

print("Model experiments notebook ready for execution!")